# dowload models

In [10]:
# download text detection model

!wget -P assets/ https://www.dropbox.com/s/r2ingd0l3zt8hxs/frozen_east_text_detection.tar.gz
!tar -xvf assets/frozen_east_text_detection.tar.gz -C assets/

# download text recognition model
!wget -P assets/ https://www.dropbox.com/sh/j3xmli4di1zuv3s/AABzCC1KGbIRe2wRwa3diWKwa/None-ResNet-None-CTC.pth

--2025-03-12 12:17:38--  https://www.dropbox.com/s/r2ingd0l3zt8hxs/frozen_east_text_detection.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:601d:18::a27d:512, 162.125.5.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:601d:18::a27d:512|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/cglz32ztcddhp7bfr6uml/frozen_east_text_detection.tar.gz?rlkey=wdrhf4g6h004sa09oexkyg86v [following]
--2025-03-12 12:17:39--  https://www.dropbox.com/scl/fi/cglz32ztcddhp7bfr6uml/frozen_east_text_detection.tar.gz?rlkey=wdrhf4g6h004sa09oexkyg86v
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5258cb6a1fd8ac6c64efd96c29.dl.dropboxusercontent.com/cd/0/inline/ClsCU24vm66zlDeBO3Crm3IGMv0jZPuIGzlPGLoXp2I4IUA4IDwsyIcgErtCcPs5H80fsvURKZvPhg0VjdDld-TGiG8TczjGk-VGrZFW423jDFS8m-yijACxnQTyjQ1Ygww/file# [following]
--2025-03-12 12:17:39--  https://uc5258cb6a

## export models to ONNX format

In [11]:
!docker compose -f docker/docker-compose.yaml up tf2onnx pt2onnx

[+] Running 1/2
 ✔ Container docker-pt2onnx-1  Created                                     0.0s 
 ⠋ Container docker-tf2onnx-1  Recreate                                    0.1s 
[+] Running 1/2
 ✔ Container docker-pt2onnx-1  Created                                     0.0s 
 ⠙ Container docker-tf2onnx-1  Recreate                                    0.2s 
[+] Running 2/2
 ✔ Container docker-pt2onnx-1  Created                                     0.0s 
 ✔ Container docker-tf2onnx-1  Recreated                                   0.3s 
Attaching to pt2onnx-1, tf2onnx-1
78tf2onnx-1  | 2025-03-12 16:18:18.174663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
78

7w Enable Watch878tf2onnx-1  | 2025-03-12 16:18:18.186373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plu

## Copy to repository

In [6]:
!mv assets/detection.onnx model_repo/text_detection/1/model.onnx
!mv assets/str.onnx model_repo/text_recognition/1/model.onnx

## launching the server

In [12]:
!docker compose -f docker/docker-compose.yaml up tritonserver

[+] Running 0/1
 ⠋ Container docker-tritonserver-1  C...                                   0.1s 
[+] Running 1/1
 ✔ Container docker-tritonserver-1  C...                                   0.1s 
Attaching to tritonserver-1
Gracefully stopping... (press Ctrl+C again to force)
Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint docker-tritonserver-1 (daa02686779d676a8006ee0c4c638cab88b837eea521ce477973686853ea3fb4): Bind for 0.0.0.0:8000 failed: port is already allocated


## Test client

In [32]:
import cv2
import tritonclient.http as httpclient
from utils.infer_functions import detection_preprocessing, detection_postprocessing, recognition_postprocessing
SAVE_INTERMEDIATE_IMAGES = False



In [33]:
img_path = "img3.png"

client = httpclient.InferenceServerClient(url="localhost:8000")

raw_image = cv2.imread(img_path)
preprocessed_img = detection_preprocessing(raw_image)

detection_input = httpclient.InferInput(
    "input_images:0", preprocessed_img.shape, datatype="FP32"
)

detection_input.set_data_from_numpy(preprocessed_img, binary_data=True)

detection_response = client.infer(
    model_name="text_detection",
    inputs=[detection_input],
)

scores = detection_response.as_numpy("feature_fusion/Conv_7/Sigmoid:0")
geometry = detection_response.as_numpy("feature_fusion/concat_3:0")

In [34]:
cropped_images = detection_postprocessing(scores, geometry, preprocessed_img)

In [35]:
cropped_images.shape

(1, 1, 32, 100)

In [36]:
cropped_images = detection_postprocessing(scores, geometry, preprocessed_img)
recognition_input = httpclient.InferInput("input.1", cropped_images.shape, datatype="FP32")
recognition_input.set_data_from_numpy(cropped_images, binary_data=True)

recognition_response = client.infer(
    model_name="text_recognition",
    inputs=[recognition_input],
)

final_text = recognition_postprocessing(recognition_response.as_numpy("308"))
print(final_text)

road
